In [3]:
import pandas as pd
import requests
from IPython.core.display import clear_output
output_path = 'D:\\Data\\Box-Office-Forecasting'
import os 

## Screen Counts

In [43]:
df_columns = ['title', 'studio', 'total_gross', 'total_theaters', 'opening_gross', 'opening_theaters', 'opening_date']

In [44]:
letters = ['#', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 
               'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

all_data = pd.DataFrame(columns = df_columns)
for letter in letters:
    page = 1
    fetch_complete = False
    while not fetch_complete:
        url = "http://www.boxofficemojo.com/movies/alphabetical.htm?letter=" + letter + "&p=.htm&page=" + str(page)
        print(url)
        clear_output(wait = True)
        
        req = requests.get(url=url)
        content = pd.read_html(req.content) # pandas uses beautifulsoup to parse the html content

        tabular_data = content[3] # choose the correct table from the result
        tabular_data = tabular_data.drop(0) # drop the row with the title
        if len(tabular_data) == 0:
            fetch_complete = True
            break
        tabular_data.columns = df_columns # housekeeping renamer
        
        all_data = pd.concat([all_data, tabular_data])
        if letter == '#': # it has only one page
            fetch_complete = True
        page = page + 1
    print("Letter {0} is complete".format(letter))
    clear_output(wait = True)

Letter Z is complete


In [48]:
all_data.to_csv(os.path.join(output_path, 'mojo-stats.csv'), sep=';', encoding= 'utf8', index=False)

## Seasonality

In [ ]:
frames = []
for year in range(2000, 2019, 1):
    url ='https://www.boxofficemojo.com/seasonal/?chart=byyear&yr='+str(year)+'&view=releasedate'

    # pretend to be browser
    header = {
      "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
      "X-Requested-With": "XMLHttpRequest"
    }

    r = requests.get(url, headers=header)
    data = pd.read_html(r.text, header=0)[2].rename(index=str, columns={"Season": "season", "Avg..1": "daily_avg_rev"})
    data = data.dropna() # remove last row(Totals) and multiple season rows
    data = data[['season', 'daily_avg_rev']]
    data['year'] = year

    frames.append(data)

In [138]:
all_data = pd.concat(frames)
all_data['daily_avg_rev'] = all_data['daily_avg_rev'].apply(lambda x: float(x.replace('$', '')))
all_data['year_season_pct'] = all_data.groupby('year')['daily_avg_rev'].apply(lambda x: x.astype(float)/x.sum())

In [139]:
all_data.to_csv(os.path.join(output_path, 'seasonality-yearly.csv'), sep=';', encoding= 'utf8', index=False)

In [151]:
all_data['overall_season_pct_All'] = all_data.groupby('season')['daily_avg_rev'].transform(lambda x: x.sum()) / all_data['daily_avg_rev'].sum()
sample = all_data[all_data.year >= 2010]
sample['overall_season_pct'] = sample.groupby('season')['daily_avg_rev'].transform(lambda x: x.sum()) / sample['daily_avg_rev'].sum()
sample = sample[['season', 'overall_season_pct']].drop_duplicates()

C:\Users\burak\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [153]:
sample.to_csv(os.path.join(output_path, 'seasonality-overall.csv'), sep=';', encoding= 'utf8', index=False)